# Thesis: Training an Adapter for Cruise

This notebook documents the workflow for training a YOLO-based adapter model tailored for cruise applications. The process includes dataset preparation, configuration file creation, model training, and result management.

## Install Required Libraries

In this step, we will install the necessary libraries for training and evaluation. This includes the `ultralytics` package, which provides the YOLO implementation used in this workflow.

In [ ]:
!pip install -q ultralytics
from ultralytics import YOLO
import os
import cv2
import torch
import matplotlib.pyplot as plt
from pathlib import Path

## Create YAML Configuration for Training

This section describes how to automatically generate a `data.yaml` configuration file required for YOLO training. The script reads class names from `classes.txt`, sets up dataset paths, and writes the configuration in YAML format.

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/kaggle/input/adapter-cruise-control-dataset',
      'train': 'train/images',
      'val': 'valid/images',
      'test': 'test/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/kaggle/input/adapter-cruise-control-dataset/classes.txt'
path_to_data_yaml = 'data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat data.yaml

# Data Visualization

This section demonstrates how to visualize the training data with bounding boxes. The code will:

1. Load a random image from the training dataset
2. Read its corresponding label file
3. Draw bounding boxes and class labels on the image
4. Display the annotated image using matplotlib

This visualization helps verify that:
- Images are loading correctly
- Label files are properly formatted
- Bounding box coordinates are accurate
- Class IDs are valid

The visualization uses:
- OpenCV for image processing and drawing
- Matplotlib for display
- Green bounding boxes with class labels
- RGB color format for proper display

You can run the next cell to see a random training example with its annotations.


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def visualize_random_sample(data_path, num_samples=1):
    """
    Visualize random samples from the dataset with bounding boxes.
    
    Args:
        data_path (str): Path to the dataset directory
        num_samples (int): Number of random samples to visualize
    """
    # Load the images and labels
    images_path = os.path.join(data_path, 'train', 'images')
    labels_path = os.path.join(data_path, 'train', 'labels')
    
    # Get list of image files
    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Select random images
    selected_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    for random_image in selected_images:
        image_path = os.path.join(images_path, random_image)
        label_path = os.path.join(labels_path, random_image.replace('.jpg', '.txt')
                                 .replace('.jpeg', '.txt').replace('.png', '.txt'))
        
        # Read and display the image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Read labels and draw boxes
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                
            height, width = img.shape[:2]
            for line in lines:
                class_id, x_center, y_center, w, h = map(float, line.strip().split())
                
                # Convert normalized coordinates to pixel coordinates
                x1 = int((x_center - w/2) * width)
                y1 = int((y_center - h/2) * height)
                x2 = int((x_center + w/2) * width)
                y2 = int((y_center + h/2) * height)
                
                # Draw rectangle
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Add class label
                cv2.putText(img, f'Class {int(class_id)}', (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.title(f'Random Training Image with Bounding Boxes: {random_image}')
        plt.axis('off')
        plt.show()
        
        # Print image details
        print(f'Image shape: {img.shape}')
        print(f'Image path: {image_path}')
        print(f'Number of objects detected: {len(lines) if os.path.exists(label_path) else 0}\n')

# Example usage
data_path = '/kaggle/input/adapter-cruise-control-dataset'
visualize_random_sample(data_path, num_samples=1)


## Start YOLO model training

The model and training parameters are defined in the cell below.
Please run the next cell to begin training.

In [ ]:
# !yolo task=detect mode=train model=yolo11s.pt data=data.yaml epochs=120 imgsz=640 device=0,1 patience=10

# Load pretrained model (better starting point than from scratch)
model = YOLO("yolo11n.pt")  # or "yolov8s.pt" for standard YOLOv8

In [ ]:
# Train the model with optimized parameters
model.train(
    data="data.yaml",
    epochs=300,
    imgsz=640,
    batch=16,  # Adjust based on your GPU memory
    device=[0,1],  # Use both GPUs
    patience=30,  # Early stopping if no improvement for 30 epochs
    optimizer='auto',  # Let YOLO choose the best optimizer
    lr0=0.01,  # Initial learning rate
    lrf=0.01,  # Final learning rate
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=7.5,  # box loss gain
    cls=0.5,  # cls loss gain
    dfl=1.5,  # dfl loss gain
    hsv_h=0.015,  # image HSV-Hue augmentation
    hsv_s=0.7,  # image HSV-Saturation augmentation
    hsv_v=0.4,  # image HSV-Value augmentation
    degrees=0.0,  # image rotation
    translate=0.1,  # image translation
    scale=0.5,  # image scale
    shear=0.0,  # image shear
    perspective=0.0,  # image perspective
    flipud=0.0,  # image flip up-down
    fliplr=0.5,  # image flip left-right
    mosaic=1.0,  # image mosaic
    mixup=0.0,  # image mixup
    copy_paste=0.0  # segment copy-paste
)

## Training Metrics Analysis

After training, we can analyze the model's performance metrics to understand its effectiveness. The following metrics are particularly important:

### Key Performance Indicators
- **mAP (mean Average Precision)**: Overall detection accuracy
- **Precision**: Ratio of true positives to all detections
- **Recall**: Ratio of true positives to all ground truth objects
- **F1-Score**: Harmonic mean of precision and recall

### Training Progress
- **Loss Curves**: Monitor training and validation loss
- **Learning Rate**: Track learning rate adjustments
- **Confusion Matrix**: Analyze detection errors

### Model Efficiency
- **Inference Speed**: Frames per second (FPS)
- **Model Size**: Memory footprint
- **FLOPs**: Computational complexity

The metrics will be visualized in the next cell to help evaluate the model's performance.


In [ ]:
# Run validation and get detailed metrics
metrics = model.val()

# Extract and print key performance metrics
print("\n=== Model Performance Metrics ===")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.precision:.4f}")
print(f"Recall: {metrics.box.recall:.4f}")
print(f"F1-Score: {metrics.box.f1:.4f}")

# Print per-class metrics
print("\n=== Per-Class Metrics ===")
for i, cls in enumerate(model.names):
    print(f"{cls}:")
    print(f"  Precision: {metrics.box.precision_per_class[i]:.4f}")
    print(f"  Recall: {metrics.box.recall_per_class[i]:.4f}")
    print(f"  F1-Score: {metrics.box.f1_per_class[i]:.4f}")

# Calculate and print inference speed
print("\n=== Inference Speed ===")
print(f"Average inference time: {metrics.speed['inference']:.2f} ms")
print(f"FPS: {1000/metrics.speed['inference']:.1f}")

# ## Test Results Analysis

# ### Test Images Directory Structure



In [ ]:
# Load the trained model
model = YOLO('runs/detect/train/weights/best.pt')

# Define test images directory
test_dir = Path('test_images')
if not test_dir.exists():
    print(f"Test directory {test_dir} not found!")
    exit()

# Get all image files
image_files = []
for ext in ['*.jpg', '*.jpeg', '*.png']:
    image_files.extend(list(test_dir.glob(ext)))

# Create a figure with subplots
n_images = len(image_files)
n_cols = 3
n_rows = (n_images + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten()

# Process each image
for idx, img_path in enumerate(image_files):
    if idx >= len(axes):
        break
        
    # Read and process image
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model(img)
    
    # Get the first result (since we're processing single images)
    result = results[0]
    
    # Draw boxes on the image
    annotated_img = result.plot()
    
    # Display image
    axes[idx].imshow(annotated_img)
    axes[idx].set_title(f'Detections: {len(result.boxes)}')
    axes[idx].axis('off')

# Hide empty subplots
for idx in range(len(image_files), len(axes)):
    axes[idx].axis('off')
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

# Print detection statistics
print("\n=== Detection Statistics ===")
total_detections = 0
class_counts = {}

for img_path in image_files:
    results = model(img_path)
    result = results[0]
    
    # Count detections per class
    for box in result.boxes:
        cls = int(box.cls[0])
        cls_name = model.names[cls]
        class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
        total_detections += 1

print(f"Total images processed: {len(image_files)}")
print(f"Total detections: {total_detections}")
print("\nDetections per class:")
for cls_name, count in class_counts.items():
    print(f"{cls_name}: {count}")



## Copy Training Results to Save Server

This section demonstrates how to securely copy the `runs` directory containing training results to a remote save server. This ensures that your experiment outputs are backed up and accessible for further analysis or sharing.

In [ ]:
!pip install -q gdown
!gdown 'https://drive.google.com/uc?id=1nQ0_w3uG8McFgPxt-kVS1RPW1aKpb2YS'
!chmod 400 /kaggle/working/gcp-key
!ssh -i /kaggle/working/gcp-key -o StrictHostKeyChecking=no trung@34.142.148.134 "rm -rf /home/trung/runs"
!scp -i /kaggle/working/gcp-key -o StrictHostKeyChecking=no -r runs trung@34.142.148.134:/home/trung/
!echo "Done!"

In [ ]:
!zip -r runs.zip runs